In [1]:
import pandas as pd
from custom_func import *

In [2]:
df_P02_006 = pd.read_excel(f"{INPUTS_PATH}/P2/P02_006.xlsx")
df_P02_007 = pd.read_excel(f"{INPUTS_PATH}/P2/P02_007_disposers_timecreate.xlsx")

In [3]:
colnames = {
    "Область": "region",
    "Податки на одну особу (p2_01)":'p2_01_raw',
    "Дохід без міжбюдж. трансфертів (p2_02)":'p2_02_raw',
    "Дохід без міжбюдж. трансфертів порівняно з минулим періодом (p2_03)": "p2_03_raw",
    "Капітальні видатки (p2_04)":'p2_04_raw',
    "Плата за землю (p2_05)":'p2_05_raw'
}

df_P02_006 = pd.read_excel(f"{INPUTS_PATH}/P2/P02_006.xlsx").rename(columns=colnames)

In [4]:
normalize_parameter(df_P02_006, "p2_01_raw", "p2_01")
normalize_parameter(df_P02_006, "p2_02_raw", "p2_02")
normalize_parameter(df_P02_006, "p2_03_raw", "p2_03")
normalize_parameter(df_P02_006, "p2_04_raw", "p2_04")
normalize_parameter(df_P02_006, "p2_05_raw", "p2_05")

df_P02_006_fin = df_P02_006.loc[:, df_P02_006.columns.str.contains("region|^p")]

Емпіричні границі: (187.8519975209227, 1053.401566199802)
Коефіцієнти нормалізації: (0.0011553353339732323, -0.21703205029337405)
Параметр p2_01 нормалізовано і додано до таблиці

Емпіричні границі: (489036160.8999999, 4323495164.98)
Коефіцієнти нормалізації: (2.607929825135605e-10, -0.12753719895809246)
Параметр p2_02 нормалізовано і додано до таблиці

Емпіричні границі: (0.5539636554771445, 1.046366956816354)
Коефіцієнти нормалізації: (2.030855595972364, -1.1250201896910659)
Параметр p2_03 нормалізовано і додано до таблиці

Емпіричні границі: (149620804.27, 1420569757.0)
Коефіцієнти нормалізації: (7.868136622261647e-10, -0.11772369295290289)
Параметр p2_04 нормалізовано і додано до таблиці

Емпіричні границі: (28855748.39, 452003346.2500001)
Коефіцієнти нормалізації: (2.36324158534123e-09, -0.06819310457139124)
Параметр p2_05 нормалізовано і додано до таблиці



---

In [5]:
# createDate
df_P02_007_fin = df_P02_007.loc[df_P02_007["status"].eq("complete")].copy()

df_P02_007_fin["Чи відкриті торги?"] = df_P02_007_fin["procurementMethod"].eq("open").astype(int)
open_procur = df_P02_007_fin.loc[df_P02_007_fin['Чи відкриті торги?']==1].groupby('region')["tenderValue"].sum().rename('open')
all_procur = df_P02_007_fin.groupby('region')["tenderValue"].sum().rename('all')
df_P02_007_fin = pd.concat([open_procur, all_procur], axis=1).reset_index()
df_P02_007_fin["p2_06_raw"] = df_P02_007_fin["open"] / df_P02_007_fin["all"]

In [6]:
normalize_parameter(df_P02_007_fin, "p2_06_raw", "p2_06", min_bound=0)

Емпіричні границі: (0, 0.9640196712048119)
Коефіцієнти нормалізації: (1.0373232309152163, -0.0)
Параметр p2_06 нормалізовано і додано до таблиці



In [7]:
#Група 1: p2_01-p2_05, показники бюджету
#Група 2: p2_06, публічні закупівлі
#Група 3: p2_07-p2_10, аудиторські показники

dict_weights = {
    'p2_01':0.4,
    'p2_02':0.4,
    'p2_03':0.4,
    'p2_04':0.4,
    'p2_05':0.4,
    'p2_06':1,
    'p2_07':0.25,
    'p2_08':0.25,
    'p2_09':0.25,
    'p2_10':0.25
}

sources = [df_P02_006_fin, df_P02_007_fin]
save_data(sources,regions,dict_weights,'P2')

Дані за галузевим параметром P2 збережено
               region     p2_01     p2_02     p2_03     p2_04     p2_05  \
0           Волинська  0.599943  0.160247  0.637031  0.073066  0.072148   
1           Вінницька  0.874382  0.465336  0.919764  0.267598  0.302305   
2    Дніпропетровська  0.717872  1.000000  0.081421  1.000000  1.000000   
3            Донецька  0.221649  0.437363  0.204585  0.504806  0.216313   
4         Житомирська  0.766951  0.294074  0.752153  0.076793  0.158528   
5        Закарпатська  0.347174  0.115136  0.281686  0.120125  0.028436   
6          Запорізька  0.699201  0.425825  0.276226  0.172996  0.329854   
7            Київська  0.839828  0.575290  0.274995  0.726074  0.335364   
8   Івано-Франківська  0.649869  0.285045  0.972227  0.274086  0.201656   
9      Кіровоградська  0.475612  0.105768  0.156829  0.001454  0.163774   
10          Луганська  0.000000  0.013674  0.196219  0.049713  0.001341   
11          Львівська  0.501859  0.516078  0.102712  0.607